<a href="https://colab.research.google.com/github/ehdghks316/itwill_studyGroup_D/blob/main/5_20%ED%8C%8C%EC%9D%B4%EC%8D%AC(%EC%A3%BC%EC%84%B1%EB%B6%84%EB%B6%84%EC%84%9D).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
차원축소(dimension reduction)
- 변수를 줄이는 것

고전, 국어, 프랑스어, 수학, 물리, IQ(정답)
# 변수들이 많다고 예측을 잘 하는것이 아니다

고전,국어,프랑스 -> 언어지능
수학,물리 -> 수리지능

- 상관계수가 높은 변수들 끼리 묶어 보는게 중요하다.
#공분산은 2개의 확률변수간의 선형관계, 공분산을 표준화하는게 상관계수

import pandas as pd
import matplotlib.pyplot as plt
iris = pd.read_csv('c:/data/iris.csv')
iris.head()

# 그래프 그려보기
markers = ['^','s','o']

for i,marker in enumerate(markers):
    x_data = iris[iris['Name'] == iris.Name.unique()[i]]['SepalLength']
    y_data = iris[iris['Name'] == iris.Name.unique()[i]]['SepalWidth']
    plt.scatter(x_data,y_data,marker=marker, label=iris.Name.unique()[i])

plt.legend()
plt.xlabel('SepalLength')
plt.ylable('SepalWidth')
plt.show()

markers = ['^','s','o']

for i,marker in enumerate(markers):
    x_data = iris[iris['Name'] == iris.Name.unique()[i]]['PetalLength']
    y_data = iris[iris['Name'] == iris.Name.unique()[i]]['PetalWidth']
    plt.scatter(x_data,y_data,marker=marker, label=iris.Name.unique()[i])

plt.legend()
plt.xlabel('PetalLength')
plt.ylabel('PetalWidth')
plt.show()

# 표준화
from sklearn.preprocessing import StandardScaler
iris_s = StandardScaler().fit(iris.iloc[:,:-1])
iris_s.mean_
iris_s.scale_

iris_scaled = iris_s.transform(iris.iloc[:,:-1])
iris_scaled
iris_scaled.shape

#주성분 분석, PCA(Principal Component Analysis)
from sklearn.decomposition import PCA
pca = PCA(n_components=2) # 하이퍼파라미터 값( 몇 개로 성분을 분류할 것인가)
pca.fit(iris_scaled)

print(pca.explained_variance_ratio_) #설명력을 출력하는 함수, pca_component_1이 72%로 설명할 수 있고 pca_component_2는 23%로 설명할 수 있다. 첫번째가 설명력이 좋다고 볼 수 있다.

iris_pca = pca.transform(iris_scaled)
iris_pca # 4개의 피쳐를 2개의 피쳐로 변환 함, 첫번째의 열을 기준으로 보는 것이 좋다.

pca_columns = ['pca_component_1','pca_component_2']
iris_pca = pd.DataFrame(iris_pca,columns=pca_columns)
iris_pca['Name'] = iris.Name
iris_pca # 2개의 피처로 name을 분류하는 지

markers = ['^','s','o']

for i,marker in enumerate(markers):
    x_data = iris_pca[iris_pca['Name'] == iris_pca.Name.unique()[i]]['pca_component_1']
    y_data = iris_pca[iris_pca['Name'] == iris_pca.Name.unique()[i]]['pca_component_2']
    plt.scatter(x_data,y_data,marker=marker, label=iris_pca.Name.unique()[i])

plt.legend()
plt.xlabel('pca_component_1')
plt.ylabel('pca_component_2')
plt.show()

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score # 
rfc = RandomForestClassifier()
score = cross_val_score(rfc,iris.iloc[:,:-1],iris.Name,scoring='accuracy',cv=3)
score # 학습완료 됨, iris.Name.unique() 순서대로 나옴

model = RandomForestClassifier(criterion='entropy',n_estimators=100,oob_score=True)
model.fit(iris.iloc[:,:-1],iris.Name)

iris.iloc[149]

y_pred = model.predict([[5.9,3.0,5.1,1.8]])
y_pred

# 주성분분석으로 나온 것도 체크해보기 
rfc = RandomForestClassifier()
score = cross_val_score(rfc,iris_pca.iloc[:,:-1],iris_pca.Name,scoring='accuracy',cv=3)
score # 학습완료 됨, iris.Name.unique() 순서대로 나옴, 주성분분석 안 했을 때보다 더 낮게 나옴 그래서 무조건 주성분 분석이 좋은 것이 아니다라는 것

model = RandomForestClassifier(criterion='entropy',n_estimators=100,oob_score=True)
model.fit(iris_pca.iloc[:,:-1],iris_pca.Name)

1. 새로운 값을 표준화 수행
test = [[5.9,3.0,5.1,1.8]] # scaling을 해야함
test_s = iris_s.transform(test)  # 표준화
test_s

2. 표준화 데이터를 pca값으로 변환
test_pca = pca.transform(test_s)
test_pca

3. 새로운 pca값으로 모델에 적용
y_pred = model.predict(test_pca) # 
y_pred


Autoencoder
- 비지도학습(unsupervisor)
- 차원축소된 이미지가 나옴
- 오토인코더는 입력값과 출력값을 같게 하는 신경망
- encoder : 입력값을 받아 특징값으로 변환하는 신경망
- latent : 신경망 내부에서 추출된 특징되는 값들이 모여있는 곳
- decoder : 특징값을 출력값으로 변환하는 신경망
# 화질이 않좋은 이미지를 좋게 만드는 작업도 함
# cnn에서 적용해보기

from tensorflow.keras.layers import Dense,Input,Conv2D,Conv2DTranspose, Flatten,Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import datasets
(x_train,y_train),(x_test,y_test) = datasets.mnist.load_data() # 정답라벨인 y는 필요없음
x_train.shape
x_train = x_train.astype('float32')/255 # scaling
x_test = x_test.astype('float32')/255 # scaling

▶방법1
Inputs = Input(shape=(28,28,1))
x = Flatten()(Inputs)
latent = Dense(10,activation='relu')(x)
x = Dense(784)(latent)
x = Reshape((28,28))(x)
model = Model(Inputs,x)
model.summary()

model.compile(optimizer='adam',loss='mse')
model.fit(x_train,x_train,epochs=10,batch_size=32,validation_data=(x_test,x_test)) # batch_size는 8~64 그이상은 x

predict_img = model.predict(x_test)

n = 10
plt.figure(figsize=(20,4))
for i in range(n):
    ax = plt.subplot(2,n,i+1)
    plt.imshow(x_test[i])
    plt.title('original')
    plt.gray()
    ax.get_xaxis().set_visible(False) # x축 출력x
    ax.get_xaxis().set_visible(False) # y축 출력 x 
    
    ax = plt.subplot(2,n,i+1+n)
    plt.imshow(predict_img[i])
    plt.title('autoencoder')
    plt.gray()
    ax.get_xaxis().set_visible(False) # x축 출력x
    ax.get_xaxis().set_visible(False) # y축 출력 x 


▶방법2
Inputs = Input(shape=(28,28,1))
x = Conv2D(32,3,2,activation='relu',padding='same')(Inputs)
x = Conv2D(64,3,2,activation='relu',padding='same')(x)
x = Flatten()(x)
latent = Dense(10)(x)
x = Dense(7*7*64)(latent)
x = Reshape((7,7,64))(x)
x = Conv2DTranspose(64,3,2,activation='relu',padding='same')(x)
x = Conv2DTranspose(32,3,2,activation='relu',padding='same')(x)
x = Conv2DTranspose(1,3,1,activation='relu',padding='same')(x) 
model = Model(Inputs,x)
model.summary() # 모델은 계속 체크해보기


model.compile(optimizer='adam',loss='mse')
model.fit(x_train,x_train,epochs=10,batch_size=32,validation_data=(x_test,x_test)) # batch_size는 8~64 그이상은 x

predict_img = model.predict(x_test)

n = 10
plt.figure(figsize=(20,4))
for i in range(n):
    ax = plt.subplot(2,n,i+1)
    plt.imshow(x_test[i])
    plt.title('original')
    plt.gray()
    ax.get_xaxis().set_visible(False) # x축 출력x
    ax.get_xaxis().set_visible(False) # y축 출력 x 
    
    ax = plt.subplot(2,n,i+1+n)
    plt.imshow(predict_img[i])
    plt.title('autoencoder')
    plt.gray()
    ax.get_xaxis().set_visible(False) # x축 출력x
    ax.get_xaxis().set_visible(False) # y축 출력 x 


▶방법3
encoder_inputs = Input(shape=(28,28,1))
x = Conv2D(32,3,1,activation='relu',padding='same')(encoder_inputs)
x = Conv2D(64,3,1,activation='relu',padding='same')(x)
x = Flatten()(x)
encoder_output = Dense(2)(x)
encoder = Model(encoder_inputs,encoder_output)
encoder.summary()

decoder_inputs = Input(shape=(2,))
x = Dense(7*7*64)(decoder_inputs)
x = Reshape((7,7,64))(x)
x = Conv2DTranspose(64,3,2,activation='relu',padding='same')(x)
x = Conv2DTranspose(32,3,2,activation='relu',padding='same')(x)
decoder_output = Conv2DTranspose(1,3,1,activation='relu',padding='same')(x) 
decoder = Model(decoder_inputs,decoder_output)
decoder.summary() # 모델은 계속 체크해보기

# encoder 모델과 decoder 모델을 연결하는 작업
encoder_in = Input(shape=(28,28,1))
x = encoder(encoder_in)
decoder_out = decoder(x)

model_auto = Model(encoder_in,decoder_out)
model_auto.summary()

model_auto.compile(optimizer='adam',loss='mse')
model_auto.fit(x_train,x_train,epochs=10,batch_size=32,validation_data=(x_test,x_test)) # batch_size는 8~64 그이상은 x

x_img = encoder.predict(x_test)

plt.figure(figsize=(15,12))
plt.scatter(x=x_img[:,0],y=x_img[:,1],c=y_test,cmap=plt.get_cmap('Paired'),s=4)
plt.colorbar()
plt.show()

통계에서 가장 중요한 것은 분산!

cnn에서 convolution층에서 특징을 추출하는 것


